<a href="https://colab.research.google.com/github/GuiXu40/deeplearning0/blob/main/Basic_code/Dlow%E6%BA%90%E7%A0%81%E5%88%86%E6%9E%90/002_exp_vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. exp_vae.py 理解
该文件是用vae模型预处理的训练文件。

In [ ]:
import os # 文件操作
import sys # 程序与python解释器的交互
import math
import pickle # 序列化对象
import argparse # 解析命令行参数
import time
from torch import optim
from torch.utils.tensorboard import SummaryWriter # 可视化工具

sys.path.append(os.getcwd())
from utils import *
from motion_pred.utils.config import Config
from motion_pred.utils.dataset_h36m import DatasetH36M
from motion_pred.utils.dataset_humaneva import DatasetHumanEva
from models.motion_pred import *

# 损失函数
def loss_function(X, Y_r, Y, mu, logvar):
    MSE = (Y_r - Y).pow(2).sum() / Y.shape[1]
    MSE_v = (X[-1] - Y_r[0]).pow(2).sum() / Y.shape[1]
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / Y.shape[1]
    loss_r = MSE + cfg.lambda_v * MSE_v + cfg.beta * KLD
    return loss_r, np.array([loss_r.item(), MSE.item(), MSE_v.item(), KLD.item()])


def train(epoch):
    """
      epoch: 训练批次
    """
    t_s = time.time() # 计算时间
    train_losses = 0  # 终loss
    total_num_sample = 0 # 总的采样数
    loss_names = ['TOTAL', 'MSE', 'MSE_v', 'KLD']
    generator = dataset.sampling_generator(num_samples=cfg.num_vae_data_sample, batch_size=cfg.batch_size) # 数据迭代器 (batch_size, T', joints_num, tra_dim)
    for traj_np in generator:
        traj_np = traj_np[..., 1:, :].reshape(traj_np.shape[0], traj_np.shape[1], -1) # traj_np: [batch_size, T', (joints_num-1)*tra_dim]
        traj = tensor(traj_np, device=device, dtype=dtype).permute(1, 0, 2).contiguous() # permute: 调整tensor维度；contiguous: 会复制一份tensor； traj: (T', batch_size, (joints_num-1)*tra_dim)
        X = traj[:t_his] # 获取历史序列
        Y = traj[t_his:] # 获取ground truth, 未来序列
        Y_r, mu, logvar = model(X, Y) # 传入模型
        loss, losses = loss_function(X, Y_r, Y, mu, logvar) # 计算损失函数
        optimizer.zero_grad() # 梯度清零
        loss.backward() # 梯度反向计算
        optimizer.step() # 更新参数
        train_losses += losses
        total_num_sample += 1 # 增加采样数

    scheduler.step() # 更新学习率
    dt = time.time() - t_s # 计算训练时间
    train_losses /= total_num_sample # 计算平均损失
    lr = optimizer.param_groups[0]['lr'] # 获取学习率
    losses_str = ' '.join(['{}: {:.4f}'.format(x, y) for x, y in zip(loss_names, train_losses)])
    logger.info('====> Epoch: {} Time: {:.2f} {} lr: {:.5f}'.format(epoch, dt, losses_str, lr)) # 输出日志， 训练批次：训练时间：损失：学习率
    for name, loss in zip(loss_names, train_losses): # 保存到tensorboard 方便可视化
        tb_logger.add_scalar('vae_' + name, loss, epoch) 


if __name__ == '__main__':

    # 1.python 中命令行参数解析
    parser = argparse.ArgumentParser() 
    parser.add_argument('--cfg', default=None)
    parser.add_argument('--mode', default='train')
    parser.add_argument('--test', action='store_true', default=False) #store_true就代表着一旦有这个参数，做出动作“将其值标为True”，也就是没有时，默认状态下其值为False 对于boolea类型的值。
    parser.add_argument('--iter', type=int, default=0)
    parser.add_argument('--seed', type=int, default=0)
    parser.add_argument('--gpu_index', type=int, default=0)
    args = parser.parse_args()

    """setup"""
    """
        随机种子
        参数类型
        训练设备
        可视化
        日志文件
    """
    np.random.seed(args.seed)
    torch.manual_seed(args.seed) # 固定随机种子，每次重新运行产生的随机数相同

    dtype = torch.float64
    torch.set_default_dtype(dtype) # 设置默认参数类型

    device = torch.device('cuda', index=args.gpu_index) if torch.cuda.is_available() else torch.device('cpu') # 选择训练设备
    if torch.cuda.is_available():
        torch.cuda.set_device(args.gpu_index)
    cfg = Config(args.cfg, test=args.test) # 获取配置类
    tb_logger = SummaryWriter(cfg.tb_dir) if args.mode == 'train' else None  # tensorboard创建，如果是train过程，就可视化
    logger = create_logger(os.path.join(cfg.log_dir, 'log.txt')) # 创建日志文件

    """parameter"""
    """
      mode: train or test
      nz: 
      t_his: 历史时间步长
      t_pred: 待预测时间步长
    """
    mode = args.mode # train or test
    nz = cfg.nz
    t_his = cfg.t_his
    t_pred = cfg.t_pred

    """data""" # 加载数据
    dataset_cls = DatasetH36M if cfg.dataset == 'h36m' else DatasetHumanEva
    dataset = dataset_cls('train', t_his, t_pred, actions='all', use_vel=cfg.use_vel) # 初始化 dataset类 加载数据
    if cfg.normalize_data:
        dataset.normalize_data() # 是否对数据进行归一化

    """model"""
    """
      model: 获取VAE模型
      optimizer: 优化器
      scheduler: 动态设置学习率
    """
    model = get_vae_model(cfg, dataset.traj_dim)
    optimizer = optim.Adam(model.parameters(), lr=cfg.vae_lr)
    scheduler = get_scheduler(optimizer, policy='lambda', nepoch_fix=cfg.num_vae_epoch_fix, nepoch=cfg.num_vae_epoch) # 设置学习率

    # 加载预训练模型
    if args.iter > 0:
        cp_path = cfg.vae_model_path % args.iter
        print('loading model from checkpoint: %s' % cp_path)
        model_cp = pickle.load(open(cp_path, "rb")) # 读取数据
        model.load_state_dict(model_cp['model_dict']) # 模型加载参数

    # 训练过程
    if mode == 'train':
        model.to(device) # 将模型移到GPU上
        model.train() # 启用 batch normalization 和 dropout
        for i in range(args.iter, cfg.num_vae_epoch):
            train(i) # 训练模型
            if cfg.save_model_interval > 0 and (i + 1) % cfg.save_model_interval == 0:
                with to_cpu(model): # 将模型移到CPU中，准备保存中间数据。
                    cp_path = cfg.vae_model_path % (i + 1) # 模型保存路径
                    model_cp = {'model_dict': model.state_dict(), 'meta': {'std': dataset.std, 'mean': dataset.mean}}
                    pickle.dump(model_cp, open(cp_path, 'wb'))





### 2. TensorBoard
https://www.cnblogs.com/sddai/p/14516691.html

### 3.如何动态学习率

### 4. 如何加载预训练模型
pickle类

### 5. 一些函数学习

In [2]:
import torch
x = torch.randn(2, 3, 5)
y = torch.permute(x, (2, 0, 1)).contiguous()
print(x.size())
print(y.size())

torch.Size([2, 3, 5])
torch.Size([5, 2, 3])


1 transpose、permute等维度变换操作后，tensor在内存中不再是连续存储的，而view操作要求tensor的内存连续存储，所以需要contiguous来返回一个contiguous copy；

2 维度变换后的变量是之前变量的浅拷贝，指向同一区域，即view操作会连带原来的变量一同变形，这是不合法的，所以也会报错；---- 这个解释有部分道理，也即contiguous返回了tensor的深拷贝contiguous copy数据；